In [1]:
from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd
from fuzzywuzzy import fuzz
import configparser
from azure.cosmos.cosmos_client import CosmosClient
from gremlin_python.driver import client, serializer
import requests
import pandas as pd
import re

In [146]:
aws_services = pd.read_csv('../data/raw/aws_services.csv')
aws_services.fillna('', inplace=True)
aws_services.head()

,category_id,category_name,icon,link,long_description,name,short_description
0,1,Analytics,,https://aws.amazon.com/athena/?c=1&pt=1,Amazon Athena is an interactive query service ...,Amazon Athena,Query Data in S3 using SQL
1,1,Analytics,,https://aws.amazon.com/cloudsearch/?c=1&pt=2,Related Links\nGet Started for Free\nAmazon Cl...,Amazon CloudSearch,Managed Search Service
2,1,Analytics,,https://aws.amazon.com/elasticsearch-service/?...,Amazon Elasticsearch Service is a fully manage...,Amazon Elasticsearch Service,Run and Scale Elasticsearch Clusters
3,1,Analytics,,https://aws.amazon.com/elasticmapreduce/?c=1&pt=4,Amazon EMR provides a managed Hadoop framework...,Amazon EMR,Hosted Hadoop Framework
4,1,Analytics,,https://aws.amazon.com/kinesis/?c=1&pt=5,"Amazon Kinesis makes it easy to collect, proce...",Amazon Kinesis,Work with Real-time Streaming Data


In [147]:
azure_services = pd.read_csv('../data/raw/azure_services.csv')
azure_services.fillna('', inplace=True)
azure_services.head()

,category_id,category_name,icon,link,long_description,name,short_description
0,Compute,Compute,https://docs.microsoft.com/en-us/azure/media/i...,https://docs.microsoft.com/en-us/azure/virtual...,"Provision virtual machines of Ubuntu, Red Hat,...",Azure Linux Virtual Machines,"Provision virtual machines of Ubuntu, Red Hat,..."
1,Compute,Compute,https://docs.microsoft.com/en-us/azure/media/i...,https://docs.microsoft.com/en-us/azure/virtual...,"Provision virtual machines for SQL Server, Sha...",Azure Windows Virtual Machines,"Provision virtual machines for SQL Server, Sha..."
2,Compute,Compute,https://docs.microsoft.com/en-us/azure/media/i...,https://docs.microsoft.com/en-us/azure/app-ser...,Quickly create powerful cloud apps for web and...,Azure App Service,Quickly create powerful cloud apps for web and...
3,Compute,Compute,https://docs.microsoft.com/en-us/azure/media/i...,https://docs.microsoft.com/en-us/azure/azure-f...,Process events with serverless code,Azure Functions,Process events with serverless code
4,Compute,Compute,https://docs.microsoft.com/en-us/azure/media/i...,https://docs.microsoft.com/en-us/azure/batch/,Cloud-scale job scheduling and compute management,Azure Batch,Cloud-scale job scheduling and compute management


In [148]:
google_services = pd.read_csv('../data/raw/google_services.csv')
google_services.fillna('', inplace=True)
google_services.head()

,category_name,icon,link,long_description,name,short_description
0,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/compute/,Google Compute Engine offers high performance ...,Compute Engine,"Scalable, high performance VMs."
1,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/appengine/,Google App Engine lets developers build scalab...,App Engine,Serverless application platform for apps and b...
2,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/kubernetes-engine/,Google Kubernetes Engine is a powerful cluster...,Kubernetes Engine,Run containerized applications.
3,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/gke-on-prem/,GKE On-Prem will provision and manage the unde...,GKE On-Prem,Make apps “cloud-ready” and move them to the c...
4,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/functions/,Google Cloud Functions makes it easy for devel...,Cloud Functions,Event-driven serverless compute platform.


In [ ]:
import concurrent.futures
from azure.cosmos.cosmos_client import CosmosClient
from gremlin_python.driver import client, serializer
from gremlin_python.structure.graph import Graph
from gremlin_python import statics
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
# from gremlin_python.process.traversal import WithOptions


import logging


class GremlinQueryManager:
    def __init__(self, account_name, master_key, database_name, graph_name):
        
        self.driver_remote_connection = DriverRemoteConnection(
            f'wss://{account_name}.gremlin.cosmosdb.azure.com:443/',
            'g',
            username=f'/dbs/{database_name}/colls/{graph_name}',
            password=master_key,
            message_serializer=serializer.GraphSONMessageSerializer()
        )

        self.g = self.setup_graph()
        
    def setup_graph(self):
        try:
            graph = Graph()
            logging.info('Trying To Login')
            g = graph.traversal().withRemote(self.driver_remote_connection)
            logging.info('Successfully Logged In')
        except Exception as e:  # Shouldn't really be so broad
            logging.error(e, exc_info=True)
            raise BadRequestError('Could not connect to Neptune')        
        
        return g

gremlin_qm = GremlinQueryManager(account_name, master_key, db_name, graph_name)
gremlin_qm.g.V('ba834b84-4fe3-40c3-94b6-991c7348db52').toList()

In [30]:
class DocumentQueryManager:
    """Utitlity class for managing cosmos db client connection
    and collections for easy querying."""
    def __init__(self, account_name, master_key, db_name):
        self.client = CosmosClient('https://{}.documents.azure.com:443/'.format(account_name), {
            'masterKey': master_key
        })
        dbs = list(self.client.ReadDatabases())
        self.db = dbs[0]['_self']
        for d in dbs:
            if d['id'] == db_name:
                self.db = d['_self']

        self.collections = {}
        for c in list(self.client.ReadContainers(self.db)):
            self.collections[c['id']] = c['_self']


class GremlinQueryManager:
    def __init__(self, account_name, master_key, database_name, graph_name):
        self.client = client.Client(
            'wss://{}.gremlin.cosmosdb.azure.com:443/'.format(account_name),
            'g',
            username="/dbs/{}/colls/{}".format(database_name, graph_name),
            password=master_key,
            message_serializer=serializer.GraphSONMessageSerializer()
        )
        
        print(self.client)
        
        self.g = self.setup_graph()
        
    def setup_graph(self):
        try:
            graph = Graph()
#             connstring = os.environ.get('GRAPH_DB')
            logging.info('Trying To Login')
            g = graph.traversal().withRemote(self.client._get_connection())
            logging.info('Successfully Logged In')
        except Exception as e:  # Shouldn't really be so broad
            logging.error(e, exc_info=True)
            raise BadRequestError('Could not connect to Neptune')
            
        print(g)
        
        
        return g

    def query(self, query):
        callback = self.client.submitAsync(query)
        return callback.result().one()
#         for r in callback.result():
#             return r
#         for f in concurrent.futures.as_completed([res.all()]):
#             print(f.result())
#         return res.done.result()
#         if callback.result():
#             res = []
#             for r in callback.result():
#                 res += r
#         return res


load_dotenv(find_dotenv())

account_name = os.environ.get('COSMOS_ACCOUNT_NAME')
db_name = os.environ.get('COSMOS_DB_NAME')
graph_name = os.environ.get('COSMOS_GRAPH_NAME')
master_key = os.environ.get('COSMOS_MASTER_KEY')

doc_qm = DocumentQueryManager(account_name, master_key, db_name)
gremlin_qm = GremlinQueryManager(account_name, master_key, db_name, graph_name)

graphtraversalsource[graph[empty]]


In [36]:
# def get_services():
abbr = 'aws'

cats = list(doc_qm.client.QueryItems(
    doc_qm.collections['ccg'],
    f"select * from c where c.label = '{abbr}_category'",
    {'enableCrossPartitionQuery': True}
))
services = list(doc_qm.client.QueryItems(
    doc_qm.collections['ccg'],
    f"select * from c where c.label = '{abbr}_service'",
    {'enableCrossPartitionQuery': True}
))

edges = list(doc_qm.client.QueryItems(
    doc_qm.collections['ccg'],
    f"select * from c where c.label = 'belongs_to'",
    {'enableCrossPartitionQuery': True}
))
cats_df = pd.DataFrame(cats)

cats_df.head()

# data_records = {}

# for s in services:
#     data_records[s['id']] = {
#         'name': extract_value(s['name']),
#         'short_description': extract_value(s['short_description']),
#         'long_description': extract_value(s['long_description']),
#     }
# return data_records

,_attachments,_etag,_rid,_self,_ts,id,label,name
0,attachments/,"""17007486-0000-0800-0000-5c44eabe0000""",taBNAIvLFShiAwAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548020414,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,"[{'_value': 'Analytics', 'id': '454af3e5-286c-..."
1,attachments/,"""17007a86-0000-0800-0000-5c44eabe0000""",taBNAIvLFShkAwAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548020414,37b89fe4-26da-4a9a-9230-967c25edb50f,aws_category,"[{'_value': 'Application Integration', 'id': '..."
2,attachments/,"""17008186-0000-0800-0000-5c44eabf0000""",taBNAIvLFShmAwAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548020415,e94f937f-5e38-4efc-825b-3a1c307be238,aws_category,"[{'_value': 'AR & VR', 'id': '3c685d37-f9aa-4d..."
3,attachments/,"""17008386-0000-0800-0000-5c44eabf0000""",taBNAIvLFShoAwAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548020415,22876f29-d576-4400-a50e-4716af003e26,aws_category,"[{'_value': 'AWS Cost Management', 'id': '6bfc..."
4,attachments/,"""17008d86-0000-0800-0000-5c44eabf0000""",taBNAIvLFShqAwAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548020415,80c596f0-5591-4264-bfbd-426b62af3ce3,aws_category,"[{'_value': 'Blockchain', 'id': '770dc6d7-ae9f..."


In [33]:
s_df = pd.DataFrame(services)
s_df.head()

,_attachments,_etag,_rid,_self,_ts,icon_uri,id,label,long_description,name,short_description,uri
0,attachments/,"""21007cbd-0000-0800-0000-5c457fc90000""",taBNAIvLFSgrEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548058569,"[{'_value': '', 'id': 'b3cb5385-7112-4f92-9be9...",f37802bd-8656-4abd-a7c2-72eb8f6419d8,aws_service,[{'_value': 'Amazon Athena is an interactive q...,"[{'_value': 'Amazon Athena', 'id': 'ee392f74-0...","[{'_value': 'Query Data in S3 using SQL', 'id'...",[{'_value': 'https://aws.amazon.com/athena/?c=...
1,attachments/,"""210082bd-0000-0800-0000-5c457fcb0000""",taBNAIvLFSgtEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548058571,"[{'_value': '', 'id': '1e346118-9b10-4472-815b...",5f10bdb8-0e7b-4341-b630-4e7864584e3e,aws_service,[{'_value': 'Related Links Get Started for Fre...,"[{'_value': 'Amazon CloudSearch', 'id': 'c28b9...","[{'_value': 'Managed Search Service', 'id': '6...",[{'_value': 'https://aws.amazon.com/cloudsearc...
2,attachments/,"""210088bd-0000-0800-0000-5c457fcc0000""",taBNAIvLFSgvEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548058572,"[{'_value': '', 'id': '438090a4-3ca3-45bd-b9ef...",cfdd7ff5-cc6e-4612-add5-aab78b38ae0c,aws_service,[{'_value': 'Amazon Elasticsearch Service is a...,"[{'_value': 'Amazon Elasticsearch Service', 'i...",[{'_value': 'Run and Scale Elasticsearch Clust...,[{'_value': 'https://aws.amazon.com/elasticsea...
3,attachments/,"""21008abd-0000-0800-0000-5c457fcc0000""",taBNAIvLFSgxEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548058572,"[{'_value': '', 'id': 'c3b474f1-d80c-4cfd-b16a...",90825abf-b36a-4201-9f3c-10c24bc30e80,aws_service,[{'_value': 'Amazon EMR provides a managed Had...,"[{'_value': 'Amazon EMR', 'id': '61d76a4d-06c7...","[{'_value': 'Hosted Hadoop Framework', 'id': '...",[{'_value': 'https://aws.amazon.com/elasticmap...
4,attachments/,"""21008cbd-0000-0800-0000-5c457fcc0000""",taBNAIvLFSgzEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,1548058572,"[{'_value': '', 'id': '0cf4bf66-59f2-4695-82b1...",9686652a-a60c-4150-b988-1beda5265373,aws_service,[{'_value': 'Amazon Kinesis makes it easy to c...,"[{'_value': 'Amazon Kinesis', 'id': 'bbf99a28-...",[{'_value': 'Work with Real-time Streaming Dat...,[{'_value': 'https://aws.amazon.com/kinesis/?c...


In [37]:
edges_df = pd.DataFrame(edges)
edges_df.head()

,_attachments,_etag,_isEdge,_rid,_self,_sink,_sinkLabel,_ts,_vertexId,_vertexLabel,id,label
0,attachments/,"""21007dbd-0000-0800-0000-5c457fca0000""",True,taBNAIvLFSgsEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,1548058570,f37802bd-8656-4abd-a7c2-72eb8f6419d8,aws_service,27c194fa-8dea-4ee3-ba8e-c83f52a95ae8,belongs_to
1,attachments/,"""210087bd-0000-0800-0000-5c457fcb0000""",True,taBNAIvLFSguEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,1548058571,5f10bdb8-0e7b-4341-b630-4e7864584e3e,aws_service,080d3407-87c9-424a-bc68-93ff9abfdd3a,belongs_to
2,attachments/,"""210089bd-0000-0800-0000-5c457fcc0000""",True,taBNAIvLFSgwEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,1548058572,cfdd7ff5-cc6e-4612-add5-aab78b38ae0c,aws_service,909f2737-b3d0-4916-925f-1a4f0fd87f6e,belongs_to
3,attachments/,"""21008bbd-0000-0800-0000-5c457fcc0000""",True,taBNAIvLFSgyEAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,1548058572,90825abf-b36a-4201-9f3c-10c24bc30e80,aws_service,f9f8868d-7184-4ad6-be13-4ca5e2df9f84,belongs_to
4,attachments/,"""21008dbd-0000-0800-0000-5c457fcc0000""",True,taBNAIvLFSg0EAAAAAAAAA==,dbs/taBNAA==/colls/taBNAIvLFSg=/docs/taBNAIvLF...,a6150c15-7343-4bec-95f8-d0c1a46cbeee,aws_category,1548058572,9686652a-a60c-4150-b988-1beda5265373,aws_service,beea6f72-a0d0-438b-b77a-2237bf500242,belongs_to


In [29]:
# q = """g.V().has('label', 'aws_service')
#         .project('name', 'short_description', 'long_description', 'category_name')
#         .by('name').by('short_description').by('long_description').by(out('belongs_to').values('name'))"""
# print(''.join(q.split()))
# len(gremlin_qm.query(q))

# q = "g.V('ba834b84-4fe3-40c3-94b6-991c7348db52').in('source_cloud')"
q = "g.V().has('label', 'aws_service')"
len(gremlin_qm.query(q))

3

In [160]:
import re
class GremlinQueryBuilder:
    """
    Basic functions to build gremlin queries that add vertices and edges
    """
    @classmethod
    def gremlin_escape(cls, s):
        return s.replace('"', '\\"').replace('$', '\\$')
    
    @classmethod
    def build_upsert_vertex_query(cls, entity_type, properties):
        q = f"""g.V().has("label", "{entity_type}"){cls.get_properties_str(properties, False)}.
                fold().
                coalesce(unfold(),
                         addV("{entity_type}"){cls.get_properties_str(properties)})"""
        return q

    @classmethod
    def build_upsert_edge_query(cls, from_id, to_id, edge_properties):
        """
        g.V().has('person','name','vadas').as('v').
           V().has('software','name','ripple').
           coalesce(__.inE('created').where(outV().as('v')),
                    addE('created').from('v').property('weight',0.5))
        """
        label = edge_properties["label"]
        return f"""g.V("{from_id}").as('v').
                    V("{to_id}").
                    coalesce(__.inE("{label}").where(outV().as('v')),
                             addE("{label}").from('v'){cls.get_properties_str(edge_properties)})"""
    
    @classmethod
    def get_by_id_query(cls, _id):
        return 'g.V("{}")'.format(_id)
    
    @classmethod
    def get_properties_str(cls, properties, create=True):
        if create:
            query_str = 'property'
        else:
            query_str = 'has'
        
    
        properties_lower = {k.lower():v for k,v in properties.items()}
        
        if "label" in properties_lower:
            del properties_lower["label"]

        output = ""
        for k, v in properties_lower.items():
            if isinstance(v, str):
                output += '.{}("{}", "{}")'.format(query_str, k, v)
            else:
                output += '.{}("{}", {})'.format(query_str, k, v)
        return output

In [122]:
test_eq = GremlinQueryBuilder.build_upsert_edge_query('4952feb6-55dc-4d8f-9d63-4fca5c4265e3', 'b8e694c2-4e0b-4c17-b01b-b00670caaa6e', {'label': 'source_cloud'})
print(test_eq)
gremlin_qm.query(test_eq)

g.V("4952feb6-55dc-4d8f-9d63-4fca5c4265e3").as('v').
                    V("b8e694c2-4e0b-4c17-b01b-b00670caaa6e").
                    coalesce(__.inE("source_cloud").where(outV().as('v')),
                             addE("source_cloud").from('v'))


[{'id': 'b0e40429-37b6-4d0b-806e-a64163d8b1ae',
  'label': 'source_cloud',
  'type': 'edge',
  'inVLabel': 'cloud',
  'outVLabel': 'aws_category',
  'inV': 'b8e694c2-4e0b-4c17-b01b-b00670caaa6e',
  'outV': '4952feb6-55dc-4d8f-9d63-4fca5c4265e3'}]

## Add clouds

In [128]:
sources = {
    'aws': 'Amazon Web Services', 
    'azure': 'Microsoft Azure', 
    'gcp': 'Google Cloud'
}
for abbreviation, source in sources.items():    
    q = GremlinQueryBuilder.build_upsert_vertex_query('cloud', {'name': source, 'abbreviation': abbreviation})
    gremlin_qm.query(q)

## Add cloud categories

In [127]:
for i, source in enumerate([aws_categories, azure_categories, google_categories]):
    abbr = list(sources.keys())[i]
    r = gremlin_qm.query(f"g.V().has('abbreviation', '{abbr}')")
    cloud_id = r[0]['id']
    
    for cat in source:
        vq = GremlinQueryBuilder.build_upsert_vertex_query(f'{abbr}_category', {'name': cat})
        v_id = gremlin_qm.query(vq)[0]['id']
        eq = GremlinQueryBuilder.build_upsert_edge_query(v_id, cloud_id, {'label': 'source_cloud'})
        gremlin_qm.query(eq)

## Add services for each cloud category

In [204]:
for abbr, df in zip(sources.keys(), [aws_services, azure_services, google_services]):
    source_name = sources[abbr]
    print(f"Adding services for {source_name}")
    def add_service_and_edge(row):
        label = f'{abbr}_service'
        props = {
            'name': row['name'],
            'short_description': row['short_description'],
            'long_description': row['long_description'],
            'uri': row['link'],
            'icon_uri': row['icon']
        }
        for k, v in props.items():
            props[k] = GremlinQueryBuilder.gremlin_escape(v)
        vq = GremlinQueryBuilder.build_upsert_vertex_query(label, props)
        
        v_res = gremlin_qm.query(vq)    

        cat_name = row['category_name']
        cat_id = gremlin_qm.query(f"g.V().has('name', '{cat_name}').has('label', '{abbr}_category')")[0]['id']

        cat_eq = GremlinQueryBuilder.build_upsert_edge_query(v_res[0]['id'], cat_id, {'label': 'belongs_to'})
        gremlin_qm.query(cat_eq)


    df.apply(add_service_and_edge, axis=1)

Adding services for Amazon Web Services
Adding services for Microsoft Azure
Adding services for Google Cloud


---
# Conflating Service Categories

In [178]:
print('AWS Categories')
aws_categories = list(aws_services['category_name'].unique())
print(aws_categories)
print()
print('Azure Categories')
azure_categories = list(azure_services['category_name'].unique())
print(azure_categories)
print()
print('Google Cloud Categories')
google_categories = list(google_services['category_name'].unique())
print(google_categories)

AWS Categories
['Analytics', 'Application Integration', 'AR & VR', 'AWS Cost Management', 'Blockchain', 'Business Applications', 'Compute', 'Customer Engagement', 'Database', 'Desktop & App Streaming', 'Developer Tools', 'Game Tech', 'Internet of Things', 'Machine Learning', 'Management & Governance', 'Media Services', 'Migration & Transfer', 'Mobile', 'Networking & Content Delivery', 'Robotics', 'Satellite', 'Security, Identity & Compliance', 'Storage']

Azure Categories
['Compute', 'Networking', 'Storage', 'Web', 'Mobile', 'Containers', 'Databases', 'Analytics', 'AI + Machine Learning', 'Internet of Things', 'Integration', 'Identity', 'Security', 'DevOps', 'Developer Tools', 'Management Tools', 'Media', 'Migration', 'Azure Stack', 'Sovereign Clouds']

Google Cloud Categories
['Compute', 'Databases', 'Management tools', 'Apigee API Platform', 'Storage', 'Migration', 'Networking', 'Developer tools', 'Cloud IoT Core', 'Anvato', 'Data analytics', 'AI and machine learning', 'Cloud IAM', '

### Resolve similar categories for AWS and Azure

In [181]:
normalized_cats = {}
for aws_cat in aws_categories:
    for azure_cat in azure_categories:
        if fuzz.ratio(aws_cat, azure_cat) > 75 or aws_cat in azure_cat or azure_cat in aws_cat:
            if len(aws_cat) < len(azure_cat):
                norm = azure_cat
            else:
                norm = aws_cat

            normalized_cats[azure_cat] = norm
            normalized_cats[aws_cat] = norm

normalized_cats

{'Analytics': 'Analytics',
 'Integration': 'Application Integration',
 'Application Integration': 'Application Integration',
 'Compute': 'Compute',
 'Databases': 'Databases',
 'Database': 'Databases',
 'Developer Tools': 'Developer Tools',
 'Internet of Things': 'Internet of Things',
 'AI + Machine Learning': 'AI + Machine Learning',
 'Machine Learning': 'AI + Machine Learning',
 'Media': 'Media Services',
 'Media Services': 'Media Services',
 'Migration': 'Migration & Transfer',
 'Migration & Transfer': 'Migration & Transfer',
 'Mobile': 'Mobile',
 'Networking': 'Networking & Content Delivery',
 'Networking & Content Delivery': 'Networking & Content Delivery',
 'Identity': 'Security, Identity & Compliance',
 'Security, Identity & Compliance': 'Security, Identity & Compliance',
 'Security': 'Security, Identity & Compliance',
 'Storage': 'Storage'}

### Resolve similar categories between Google categories and the normalized AWS/Azure categories

In [199]:
for google_cat in google_categories:
    for norm_cat in set(normalized_cats.values()):
        if fuzz.ratio(google_cat, norm_cat) > 60 or google_cat in norm_cat or norm_cat in google_cat:
            print(google_cat, norm_cat)
            normalized_cats[google_cat] = norm_cat
            
normalized_cats

Compute Compute
Databases Databases
Storage Storage
Migration Migration & Transfer
Networking Networking & Content Delivery
Developer tools Developer Tools
Data analytics Analytics
AI and machine learning AI + Machine Learning


{'Analytics': 'Analytics',
 'Integration': 'Application Integration',
 'Application Integration': 'Application Integration',
 'Compute': 'Compute',
 'Databases': 'Databases',
 'Database': 'Databases',
 'Developer Tools': 'Developer Tools',
 'Internet of Things': 'Internet of Things',
 'AI + Machine Learning': 'AI + Machine Learning',
 'Machine Learning': 'AI + Machine Learning',
 'Media': 'Media Services',
 'Media Services': 'Media Services',
 'Migration': 'Migration & Transfer',
 'Migration & Transfer': 'Migration & Transfer',
 'Mobile': 'Mobile',
 'Networking': 'Networking & Content Delivery',
 'Networking & Content Delivery': 'Networking & Content Delivery',
 'Identity': 'Security, Identity & Compliance',
 'Security, Identity & Compliance': 'Security, Identity & Compliance',
 'Security': 'Security, Identity & Compliance',
 'Storage': 'Storage',
 'Developer tools': 'Developer Tools',
 'AI and machine learning': 'AI + Machine Learning',
 'Data analytics': 'Analytics'}

In [200]:
print('AWS Categories')
aws_categories = list(aws_services['category_name'].unique())
print(aws_categories)
print()
print('Azure Categories')
azure_categories = list(azure_services['category_name'].unique())
print(azure_categories)
print()
print('Google Cloud Categories')
google_categories = list(google_services['category_name'].unique())
print(google_categories)

AWS Categories
['Analytics', 'Application Integration', 'AR & VR', 'AWS Cost Management', 'Blockchain', 'Business Applications', 'Compute', 'Customer Engagement', 'Database', 'Desktop & App Streaming', 'Developer Tools', 'Game Tech', 'Internet of Things', 'Machine Learning', 'Management & Governance', 'Media Services', 'Migration & Transfer', 'Mobile', 'Networking & Content Delivery', 'Robotics', 'Satellite', 'Security, Identity & Compliance', 'Storage']

Azure Categories
['Compute', 'Networking', 'Storage', 'Web', 'Mobile', 'Containers', 'Databases', 'Analytics', 'AI + Machine Learning', 'Internet of Things', 'Integration', 'Identity', 'Security', 'DevOps', 'Developer Tools', 'Management Tools', 'Media', 'Migration', 'Azure Stack', 'Sovereign Clouds']

Google Cloud Categories
['Compute', 'Databases', 'Management tools', 'Apigee API Platform', 'Storage', 'Migration', 'Networking', 'Developer tools', 'Cloud IoT Core', 'Anvato', 'Data analytics', 'AI and machine learning', 'Cloud IAM', '

### Let's pick up a couple obvious strays from Google

In [201]:
normalized_cats['Cloud IAM'] = normalized_cats['Identity']
normalized_cats['Cloud IOT Core'] = normalized_cats['Internet of Things']

### Add the normalized categories as super categories.
These super categories will begin to connect the services from the different cloud providers

In [203]:
for text, norm in normalized_cats.items():
    category_res = gremlin_qm.query(f"g.V().has('name', '{norm}').has('label', 'category')")
    if category_res:
        norm_category_id = category_res[0]['id']
    else:
        vq = GremlinQueryBuilder.build_upsert_vertex_query('category', {'name': norm})
        norm_category_id = gremlin_qm.query(vq)[0]['id']

    nodes = gremlin_qm.query(f"g.V().has('name', '{text}')")
    for node in nodes:        
        eq = GremlinQueryBuilder.build_upsert_edge_query(node['id'], norm_category_id, {'label': 'super_category'})
        gremlin_qm.query(eq)